In [2]:
import torch
from flask import Flask, jsonify, request
import os
import os.path
import json
import sys
import pandas as pd
import numpy as np
from operator import itemgetter
import tensorflow as tf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
#%matplotlib inline
import seaborn as sns
import time
from itertools import combinations
import time
from numpy import load ,save
physical_devices = tf.config.list_physical_devices('GPU') 
tf.config.experimental.set_memory_growth(physical_devices[0], True)

app = Flask(__name__)
trX = load('data_count.npy')
W=torch.load('count_tensor_W.pt')
hb=torch.load('count_tensor_hb.pt')
vb=torch.load('count_tensor_vb.pt')
visibleUnits=3273
customer_df = pd.read_csv('customerdata.csv',low_memory=False)
items_df = customer_df[["product_item_id", "PRODUCT_NAME", "CATEGORY"]]
items_df = items_df.drop_duplicates()
items_df = items_df.reset_index(drop = True)
items_df = pd.DataFrame(items_df.groupby('product_item_id')["PRODUCT_NAME"].apply(list)).reset_index()
purchase_df=customer_df.groupby(['CUSTOMER_ID','product_item_id']).size().reset_index(name='COUNT')
purchase_df['COUNT'] = 1
user_item_df = purchase_df.pivot(index='CUSTOMER_ID', columns='product_item_id', values='COUNT')
@app.route('/sampleapi1/<string:typeofscript>', methods = ['GET','POST'])
def disp(typeofscript):
   
    if typeofscript=='printcust1':
        start = time.time()
        user_id = request.args.get('mock_user_id')
        user_id=int(user_id)
        mock_user_id = user_id
        # customer_df[(CUSTOMER_ID[mock_user_id]]
        # customer_df.loc[mock_user_id]
        customer_df.loc[customer_df['CUSTOMER_ID'] == mock_user_id].groupby(['CUSTOMER_ID','PRODUCT_NAME']).size().reset_index(name='COUNT')

        #Selecting the input user
       
       
        inputUser = trX[mock_user_id-1].reshape(1, -1)

        inputUser = tf.convert_to_tensor(trX[mock_user_id-1],"float32")
       
        v0 = inputUser

        print(v0)
        v0.shape
       
        v0test = tf.zeros([visibleUnits], tf.float32)
        v0test.shape

        #Feeding in the user and reconstructing the input

        hh0 = tf.nn.sigmoid(tf.matmul([v0], W) + hb)

        vv1 = tf.nn.sigmoid(tf.matmul(hh0, tf.transpose(W)) + vb)

        rec = vv1

        tf.maximum(rec,1)
        for i in vv1:
            print(i)

        scored_items_df_mock = items_df[items_df['product_item_id'].isin(user_item_df.columns)]
        scored_items_df_mock = scored_items_df_mock.assign(RecommendationScore = rec[0])
        scored_items_df_mock.sort_values(["RecommendationScore"], ascending=False).head(20)

        items_df_mock = purchase_df[purchase_df['CUSTOMER_ID'] == mock_user_id]
        items_df_mock.head()

        #Merging movies_df with ratings_df by MovieID
        merged_df_mock = scored_items_df_mock.merge(items_df_mock, on='product_item_id', how='outer').drop(columns=['CUSTOMER_ID', 'COUNT'])

        final=merged_df_mock.sort_values(["RecommendationScore"], ascending=False).head(20)
       
        print(time.time(),start)
        return final.to_json(orient='records')
if __name__ == '__main__':
    app.run(debug = True,use_reloader=False,threaded=True)

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: on


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


tf.Tensor([0. 0. 0. ... 0. 0. 0.], shape=(3273,), dtype=float32)


127.0.0.1 - - [13/May/2021 15:16:41] "GET /sampleapi1/printcust1?mock_user_id=33 HTTP/1.1" 200 -


tf.Tensor(
[3.2993057e-04 1.1921624e-04 6.9280044e-04 ... 2.6421085e-05 4.5359615e-05
 3.5709061e-05], shape=(3273,), dtype=float32)
1620899201.1976683 1620899200.1259916


127.0.0.1 - - [13/May/2021 15:17:33] "GET /sampleapi/printcust?mock_user_id=33 HTTP/1.1" 404 -
127.0.0.1 - - [13/May/2021 15:17:53] "GET /sampleapi/printcust?mock_user_id=33 HTTP/1.1" 404 -
